## Supply Chain Management

## import libraries

In [95]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [10]:
features  = pd.read_csv("C:\\Users\\digit\\OneDrive\\Desktop\\Supply Chain\\features.csv")
stores = pd.read_csv("C:\\Users\\digit\\OneDrive\\Desktop\\Supply Chain\\stores.csv")
train = pd.read_csv("C:\\Users\\digit\\OneDrive\\Desktop\\Supply Chain\\train.csv")
test = pd.read_csv("C:\\Users\\digit\\OneDrive\\Desktop\\Supply Chain\\test.csv")

In [12]:
features.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True


In [14]:
stores.head(2)

,Store,Type,Size
0,1,A,151315
1,2,A,202307


In [16]:
train.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True


In [18]:
test.head(2)

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False


In [22]:
train["Store"].nunique()

45

In [24]:
stores["Store"].nunique()

45

In [26]:
len(train["Store"])

421570

In [28]:
len(stores["Store"])

45

In [20]:
data = train.merge(stores, how= 'left', on='Store')
data.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315


In [30]:
len(data["Store"])

421570

In [32]:
data.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
Type            0
Size            0
dtype: int64

In [34]:
features["Store"].nunique()

45

In [36]:
len(features["Store"])

8190

In [38]:
features.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [40]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [42]:
data = data.merge(features, how= "left", on=["Store","Date"])
data.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True


In [44]:
len(data)

421570

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday_x   421570 non-null  bool   
 5   Type          421570 non-null  object 
 6   Size          421570 non-null  int64  
 7   Temperature   421570 non-null  float64
 8   Fuel_Price    421570 non-null  float64
 9   MarkDown1     150681 non-null  float64
 10  MarkDown2     111248 non-null  float64
 11  MarkDown3     137091 non-null  float64
 12  MarkDown4     134967 non-null  float64
 13  MarkDown5     151432 non-null  float64
 14  CPI           421570 non-null  float64
 15  Unemployment  421570 non-null  float64
 16  IsHoliday_y   421570 non-null  bool   
dtypes: bool(2), float64(10), int64(3), object(2)
mem

In [48]:
data["Date"] =  pd.to_datetime(data["Date"])

In [50]:
data = data.sort_values(by= "Date")

In [52]:
data

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
277665,29,5,2010-02-05,15552.08,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False
277808,29,6,2010-02-05,3200.22,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False
277951,29,7,2010-02-05,10820.05,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False
278094,29,8,2010-02-05,20055.64,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379648,40,87,2012-10-26,24638.96,False,A,155083,49.65,3.917,3605.71,55.98,0.28,486.81,1389.06,138.728161,4.145,False
181170,19,30,2012-10-26,3740.12,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
181313,19,31,2012-10-26,3128.17,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
181599,19,33,2012-10-26,5740.14,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False


In [54]:
data.tail()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
379648,40,87,2012-10-26,24638.96,False,A,155083,49.65,3.917,3605.71,55.98,0.28,486.81,1389.06,138.728161,4.145,False
181170,19,30,2012-10-26,3740.12,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
181313,19,31,2012-10-26,3128.17,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
181599,19,33,2012-10-26,5740.14,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
421569,45,98,2012-10-26,1076.80,False,B,118221,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,False


In [56]:
data.isnull().sum().sum()

1422431

In [58]:
data.isnull().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Type                 0
Size                 0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday_y          0
dtype: int64

In [67]:
numerical_columns = data.select_dtypes(include= [np.number]).columns
numerical_columns

Index(['Store', 'Dept', 'Weekly_Sales', 'Size', 'Temperature', 'Fuel_Price',
       'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI',
       'Unemployment'],
      dtype='object')

In [73]:
data[numerical_columns] = data[numerical_columns].fillna(data[numerical_columns].median())

In [75]:
data.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday_x     0
Type            0
Size            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday_y     0
dtype: int64

In [77]:
data = data.fillna(data.mode().iloc[0])

In [79]:
data["Year"] = data["Date"].dt.year
data["Month"] = data["Date"].dt.month
data["Day"] = data["Date"].dt.day
data["Week"] = data["Date"].dt.isocalendar().week
data["DayofWeek"] = data["Date"].dt.dayofweek

In [81]:
data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,...,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Year,Month,Day,Week,DayofWeek
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,5347.45,...,1481.31,3359.45,211.096358,8.106,False,2010,2,5,5,4
277665,29,5,2010-02-05,15552.08,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4
277808,29,6,2010-02-05,3200.22,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4
277951,29,7,2010-02-05,10820.05,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4
278094,29,8,2010-02-05,20055.64,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4


In [83]:
categorical_columns= data.select_dtypes(include= ['object']).columns
categorical_columns

Index(['Type'], dtype='object')

In [85]:
if len(categorical_columns) > 0:
    data = pd.get_dummies(data, columns= categorical_columns, drop_first= True)

In [87]:
data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,...,CPI,Unemployment,IsHoliday_y,Year,Month,Day,Week,DayofWeek,Type_B,Type_C
0,1,1,2010-02-05,24924.50,False,151315,42.31,2.572,5347.45,192.0,...,211.096358,8.106,False,2010,2,5,5,4,False,False
277665,29,5,2010-02-05,15552.08,False,93638,24.36,2.788,5347.45,192.0,...,131.527903,10.064,False,2010,2,5,5,4,True,False
277808,29,6,2010-02-05,3200.22,False,93638,24.36,2.788,5347.45,192.0,...,131.527903,10.064,False,2010,2,5,5,4,True,False
277951,29,7,2010-02-05,10820.05,False,93638,24.36,2.788,5347.45,192.0,...,131.527903,10.064,False,2010,2,5,5,4,True,False
278094,29,8,2010-02-05,20055.64,False,93638,24.36,2.788,5347.45,192.0,...,131.527903,10.064,False,2010,2,5,5,4,True,False


In [89]:
x= data.drop(columns= ['Date', 'Weekly_Sales'] )
y = data['Weekly_Sales']

In [91]:
x

,Store,Dept,IsHoliday_x,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,CPI,Unemployment,IsHoliday_y,Year,Month,Day,Week,DayofWeek,Type_B,Type_C
0,1,1,False,151315,42.31,2.572,5347.45,192.00,24.60,1481.31,...,211.096358,8.106,False,2010,2,5,5,4,False,False
277665,29,5,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,131.527903,10.064,False,2010,2,5,5,4,True,False
277808,29,6,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,131.527903,10.064,False,2010,2,5,5,4,True,False
277951,29,7,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,131.527903,10.064,False,2010,2,5,5,4,True,False
278094,29,8,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,131.527903,10.064,False,2010,2,5,5,4,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379648,40,87,False,155083,49.65,3.917,3605.71,55.98,0.28,486.81,...,138.728161,4.145,False,2012,10,26,43,4,False,False
181170,19,30,False,203819,56.49,4.071,5430.75,90.07,24.60,904.34,...,138.728161,7.992,False,2012,10,26,43,4,False,False
181313,19,31,False,203819,56.49,4.071,5430.75,90.07,24.60,904.34,...,138.728161,7.992,False,2012,10,26,43,4,False,False
181599,19,33,False,203819,56.49,4.071,5430.75,90.07,24.60,904.34,...,138.728161,7.992,False,2012,10,26,43,4,False,False


In [93]:
y

0         24924.50
277665    15552.08
277808     3200.22
277951    10820.05
278094    20055.64
            ...   
379648    24638.96
181170     3740.12
181313     3128.17
181599     5740.14
421569     1076.80
Name: Weekly_Sales, Length: 421570, dtype: float64

In [97]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2, random_state= 42)

In [99]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [101]:
y_pred= model.predict(x_test)
mse= mean_squared_error(y_test, y_pred)
r2= r2_score(y_test, y_pred)

print(f"Mean Squared Error : {mse}" )
print(f"R-Squared : {r2}")

Mean Squared Error : 471988613.85739404
R-Squared : 0.09233091599074594


In [104]:
model = RandomForestRegressor()
model.fit(x_train,y_train)

y_pred= model.predict(x_test)
mse= mean_squared_error(y_test, y_pred)
r2= r2_score(y_test, y_pred)

print(f"Mean Squared Error : {mse}" )
print(f"R-Squared : {r2}")

Mean Squared Error : 11843027.910312898
R-Squared : 0.9772249796294925
